In [1]:
import numpy as np
import tensorflow as tf
import joblib
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import CarliniL2Method

# Suppress warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print("Libraries imported.")

/Users/mahima/Desktop/ENC/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mahima/Desktop/ENC/env/lib/python3.12/site-packages/art/estimators/certification/__init__.py:30: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


Libraries imported.


In [ ]:
CW_CONFIDENCE = 0.0
CW_MAX_ITER = 10
CW_BATCH_SIZE = 32

print("Loading model and data...")
model = load_model('ids_model.h5')
print("Model loaded.")
X_test = np.load('X_test_ids.npy')
y_test_ohe = np.load('y_test_ids_ohe.npy')
y_test_int = np.load('y_test_ids_int.npy')

Loading model and data...
Model loaded.


In [ ]:
label_encoder = joblib.load('label_encoder_ids.joblib')
class_names = label_encoder.classes_

print(f"Data loaded. X_test shape: {X_test.shape}")
print(f"Classes: {class_names}")

print("\nChecking baseline accuracy...")
y_pred_clean = model.predict(X_test, verbose=0)
y_pred_clean_int = np.argmax(y_pred_clean, axis=1)

clean_acc = accuracy_score(y_test_int, y_pred_clean_int)
print(f"Baseline (Clean) Accuracy: {clean_acc * 100:.2f}%")

classifier = KerasClassifier(model=model, clip_values=(X_test.min(), X_test.max()))
print("ART Classifier created.")

print("\n" + "="*50)
print(f"Starting C&W Attack (Iterations = {CW_MAX_ITER})")
print("="*50 + "\n")

attack = CarliniL2Method(
    classifier,
    confidence=CW_CONFIDENCE,
    max_iter=CW_MAX_ITER,
    batch_size=CW_BATCH_SIZE,
    verbose=True
)

n_samples = 5000
X_test_subset = X_test[:n_samples]
y_test_subset_ohe = y_test_ohe[:n_samples]
y_test_subset_int = y_test_int[:n_samples]

print(f"Generating adversarial examples on {n_samples} samples...")

X_test_adv = attack.generate(x=X_test_subset, y=y_test_subset_ohe)

print("\nEvaluating model on Adversarial Examples...")
y_pred_adv = model.predict(X_test_adv, verbose=0)
y_pred_adv_int = np.argmax(y_pred_adv, axis=1)

adv_acc = accuracy_score(y_test_subset_int, y_pred_adv_int)

Data loaded. X_test shape: (210876, 78, 1)
Classes: ['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator']

Checking baseline accuracy...
Baseline (Clean) Accuracy: 90.87%
ART Classifier created.

Starting C&W Attack (Iterations = 10)

Generating adversarial examples on 5000 samples...



C&W L_2: 100%|██████████| 157/157 [2:55:42<00:00, 67.15s/it] 



Evaluating model on Adversarial Examples...


In [5]:
print("\n" + "="*50)
print(f"RESULTS: C&W Attack (iter={CW_MAX_ITER})")
print("="*50)
print(f"Clean Accuracy (on subset): {accuracy_score(y_test_subset_int, np.argmax(model.predict(X_test_subset, verbose=0), axis=1)) * 100:.2f}%")
print(f"Attack Accuracy:            {adv_acc * 100:.2f}%")
print(f"Accuracy Drop:              {(clean_acc - adv_acc) * 100:.2f}%")
print("="*50)

print("\nClassification Report (Adversarial):")
print(classification_report(y_test_subset_int, y_pred_adv_int, target_names=class_names, zero_division=0))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_subset_int, y_pred_adv_int))

print("\n✓ C&W attack completed successfully!")


RESULTS: C&W Attack (iter=10)
Clean Accuracy (on subset): 90.56%
Attack Accuracy:            44.92%
Accuracy Drop:              45.95%

Classification Report (Adversarial):
                  precision    recall  f1-score   support

          BENIGN       0.60      0.47      0.52      2426
             Bot       0.00      0.00      0.00         9
            DDoS       0.65      0.57      0.61       560
   DoS GoldenEye       0.05      0.26      0.08        50
        DoS Hulk       0.71      0.65      0.68      1073
DoS Slowhttptest       0.08      0.34      0.14        29
   DoS slowloris       0.06      0.16      0.09        31
     FTP-Patator       0.00      0.00      0.00        41
      Heartbleed       0.00      0.00      0.00         0
    Infiltration       0.00      0.00      0.00         0
        PortScan       0.46      0.08      0.14       756
     SSH-Patator       0.01      0.16      0.02        25

        accuracy                           0.45      5000
       macro

In [ ]:
import os

print("Saving C&W adversarial data for defense models...")

file_x_adv = 'X_adv_cw_ids.npy'
file_y_int = 'y_test_cw_ids_int.npy'
file_y_ohe = 'y_test_cw_ids_ohe.npy'

np.save(file_x_adv, X_test_adv)
np.save(file_y_int, y_test_subset_int)
np.save(file_y_ohe, y_test_subset_ohe)

print(f"✓ Saved: {file_x_adv}")
print(f"✓ Saved: {file_y_int}")
print(f"✓ Saved: {file_y_ohe}")

Saving C&W adversarial data for defense models...
✓ Saved: X_adv_cw_ids.npy
✓ Saved: y_test_cw_ids_int.npy
✓ Saved: y_test_cw_ids_ohe.npy
